In [1]:
import sys
import pandas as pd
import os
import re
import glob
import shutil
import numpy as np
#import matplotlib.pyplot as plt
#import skimage
from pathlib import Path
import shutil

from base64 import b64encode
from base64 import b64decode

#plt.rcParams["figure.figsize"] = (10,10)

from IPython.core.display import display, HTML
# make the Jupyter notebook use the full screen width
display(HTML("<style>.container { width:99% !important; }</style>"))

In [18]:
#fix problem for pdf's and non-html...also for tables...

In [13]:
sys.path.append( "/notebook/nas-trainings/arne/DGFISMA/definition_extraction/" )
from cleaning import clean_html
from annotate import annotate_article, extract_definitions_and_contexts
from definition import DefinitionFinder

#small helper function
def get_article_definitions( articles, definitions  ):

    '''
    Function converts flat list of Booleans (definitions) into a list of lists, with length of each list equal to the length of each corresponding article.
     
    :param articles. List. List of Strings (articles).
    :param definitions. List. List of Booleans.
    :return List. List of Lists (Booleans)
    '''

    article_lengths=[len(article.split("\n") ) for article in articles]
    index=0
    definition_articles=[]
    for length in article_lengths:
        definition_articles.append(definitions[index: index+length ])
        index=index+length
    return definition_articles


html_file=open( "/notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/Eurlex_documents/EUR-Lex - 32015R0061 - EN - EUR-Lex.html").read()

articles=clean_html( html_file )

sentences="\n".join( articles )
sentences=sentences.split( "\n" )
definition_finder=DefinitionFinder( sentences  )
#definitions=definition_finder.get_definitions_regex()
definitions=definition_finder.get_definitions_bert( model_path='../bert_classifier/models_dgfisma_def_extraction/run_2020_06_26_11_56_31_acb319aac70b/distilbert-base-uncased_model_10.pth')
#definitions=definition_finder.get_definitions_fasttext( model_path='/notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/bootstrapped_training_set/iteration_1/model_fasttext.bin', )

definition_articles=get_article_definitions( articles, definitions )

#sanity check
assert( len( definition_articles ) == len(articles)  )

definitions_and_contexts=[]

for article, definition_article in zip( articles,  definition_articles  ):
    annotated_article=annotate_article( article.split("\n") )
    #sanity check
    assert( len(annotated_article) == len( definition_article )  )
    
    definitions_and_contexts.append( extract_definitions_and_contexts(  annotated_article, definition_article ) )
    
#extract_definitions_and_contexts does not work as we want to...
#if the context of a definition also contains a definition, we just want to add it to the context of the initially detected definition

loading ../bert_classifier/models_dgfisma_def_extraction/run_2020_06_26_11_56_31_acb319aac70b/distilbert-base-uncased_model_10.pth
tokenizing...
start inference...


In [14]:
definitions_and_contexts[-2]

[['Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to:',
  '(a)',
  'the level 1 asset amount; plus',
  '(b)',
  'the level 2A asset amount; plus',
  '(c)',
  'the level 2B asset amount;',
  'minus the lesser of:',
  '(d)',
  'the sum of (a), (b), and (c); or',
  '(e)',
  'the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex.'],
 ['‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein:',
  '(a)',
  'an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets, excluding level 1 covered bonds, that would be held by the credit institution upon the unwind of any secured funding transaction, secured lending transaction, asset exchange or collateralised derivatives transaction that matures within 30 calendar days from the calculation date and where the credit institution an

In [15]:
len(definitions_and_contexts[-2])

4

In [10]:
#open( "/notebook/nas-trainings/arne/DGFISMA/definition_extraction/notebooks/prediction.txt" ).read()
import json
MODEL_DIR="/notebook/nas-trainings/arne/DGFISMA/definition_extraction/notebooks"

with open( os.path.join( MODEL_DIR, "prediction.json" ) ) as json_file:
    config = json.load(json_file)
    
print( config['definitions'] )

‘level 1 assets’ means assets of extremely high liquidity and credit quality as referred to in the second subparagraph of Article 416(1) of Regulation (EU) No 575/2013;
‘level 2 assets’ means assets of high liquidity and credit quality as referred to in the second subparagraph of Article 416(1) of Regulation (EU) No 575/2013. level 2 assets are further subdivided into level 2A and 2B assets in accordance with Chapter 2 of Title II of this Regulation;
‘liquidity buffer’ means the amount of liquid assets that a credit institution holds in accordance with Title II of this Regulation;
‘reporting currency’ means the currency in which the liquidity items referred to in Titles II and III of Part Six of Regulation (EU) No 575/2013 must be reported to the competent authority in accordance with Article 415(1) of that Regulation;
‘asset coverage requirement’ means the ratio of assets to liabilities as determined for credit enhancement purposes in relation to covered bonds by the national law of a

In [16]:
test="In order to be eligible to form part of a credit institution's liquidity buffer, the liquid assets shall comply with each of the following requirements:■■■■(a)■■■■the general requirements laid down in Article 7;■■■■(b)■■■■the operational requirements laid down in Article 8;■■■■(c)■■■■the respective eligibility criteria for their classification as a level 1 or level 2 asset in accordance with Chapter 2."

In [25]:
import torch
torch.set_num_threads( 10 )
torch.get_num_threads()


10

In [15]:

from transformers import DistilBertConfig, DistilBertModel, BertConfig

MODEL_DIR="/notebook/nas-trainings/arne/DGFISMA/definition_extraction/bert_classifier/models_dgfisma_def_extraction/run_2020_06_26_11_56_31_acb319aac70b/"
config = BertConfig.from_json_file(os.path.join(MODEL_DIR, 'config.json' ))
type(config)

transformers.configuration_bert.BertConfig

In [16]:
config.model_type

'distilbert'

In [95]:
if type( config ) is transformers.configuration_bert.BertConfig:
    print( "s" )

NameError: name 'transformers' is not defined

In [101]:
if isinstance(config, DistilBertConfig ):
    print("sd")

In [93]:
type( config)

transformers.configuration_bert.BertConfig

In [102]:
config[ "architectures" ]

TypeError: 'BertConfig' object is not subscriptable

In [82]:
if "Distil" in config.architectures[0]:
    print("sd")

sd


In [103]:
import json

MODEL_PATH='/notebook/nas-trainings/arne/DGFISMA/definition_extraction/bert_classifier/models_dgfisma_def_extraction/run_2020_06_26_11_56_31_acb319aac70b'

with open( os.path.join( MODEL_PATH, "config.json" ) ) as json_file:
    data = json.load(json_file)

if "Distil" in "".join(data[ 'architectures' ]):
    print( "distil" )

distil


In [104]:
data['model_type']

'distilbert'

In [68]:
import json
with open('/notebook/nas-trainings/arne/DGFISMA/definition_extraction/example.json') as json_file:
    data = json.load(json_file)


decoded_content=base64.b64decode( data['content']  ).decode( 'utf-8' )
print(decoded_content)

<!DOCTYPE html>
<html class="js " xml:lang="en" prefix="eli: http://data.europa.eu/eli/ontology#" lang="en"><head>
<meta http-equiv="content-type" content="text/html; charset=UTF-8">
        <meta charset="utf-8">
        
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        
 
 
 
 
                <meta name="viewport" content="width=device-width, initial-scale=1">
            
 
	
				<script src="EUR-Lex%20-%2032015R0061%20-%20EN%20-%20EUR-Lex_files/consent.js" type="text/javascript"></script>
        
        <script type="text/javascript" src="EUR-Lex%20-%2032015R0061%20-%20EN%20-%20EUR-Lex_files/modernizr.js"></script>
        
 


 
 
 
 
 
 



 
 
 
 
 

 
 
 <title>EUR-Lex - 32015R0061 - EN - EUR-Lex</title> 
 <meta about="http://data.europa.eu/eli/reg_del/2015/61/oj" typeof="eli:LegalResource">
<meta about="http://data.europa.eu/eli/reg_del/2015/61/oj" property="eli:uri_schema" resource="http://data.europa.eu/eli/%7Btypedoc%7D/%7Byear%7D/%7Bnatural_number%7

In [120]:
clean_html( html_file[200000:309900] )

['(17)\nThe liquidity coverage ratio should apply to credit institutions both on an individual and consolidated basis, unless the competent authorities waive the application on an individual basis in accordance with Articles 8 or 10 of Regulation (EU) No 575/2013. The consolidation of subsidiary undertakings in third countries should take due account of the liquidity coverage requirements applicable in those countries. Accordingly, consolidation rules in the Union should not give a more favourable treatment to liquid assets, liquidity outflows or inflows in third country subsidiary undertakings than that which is available under the national law of those third countries.\n(18)\nIn accordance with Article 508(2) of Regulation (EU) No 575/2013, the Commission must report to the co-legislators by no later than 31 December 2015 on whether and how the liquidity coverage requirement laid down in Part Six should apply to investment firms. Until that provision starts to apply, investment firms

In [121]:
encoded_doc=b64encode(  html_file[200000:309900].encode() )
encoded_doc.decode() 


'...'

In [61]:
html_file

'<!DOCTYPE html>\n<html class="js " xml:lang="en" prefix="eli: http://data.europa.eu/eli/ontology#" lang="en"><head>\n<meta http-equiv="content-type" content="text/html; charset=UTF-8">\n        <meta charset="utf-8">\n        \n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n        \n \n \n \n \n                <meta name="viewport" content="width=device-width, initial-scale=1">\n            \n \n\t\n\t\t\t\t<script src="EUR-Lex%20-%2032015R0061%20-%20EN%20-%20EUR-Lex_files/consent.js" type="text/javascript"></script>\n        \n        <script type="text/javascript" src="EUR-Lex%20-%2032015R0061%20-%20EN%20-%20EUR-Lex_files/modernizr.js"></script>\n        \n \n\n\n \n \n \n \n \n \n\n\n\n \n \n \n \n \n\n \n \n <title>EUR-Lex - 32015R0061 - EN - EUR-Lex</title> \n <meta about="http://data.europa.eu/eli/reg_del/2015/61/oj" typeof="eli:LegalResource">\n<meta about="http://data.europa.eu/eli/reg_del/2015/61/oj" property="eli:uri_schema" resource="http://data.europa.eu/e

In [33]:
def_context=[]
for def_cont in definitions_and_contexts:
    item_join=[]
    for item in def_cont:
        item_join.append( "■■■■".join(item) )
    if item_join:
        def_context.append( "\n".join(item_join) )
def_context="\n".join( def_context )

In [34]:
print(def_context)

Covered bonds are debt instruments issued by credit institutions and secured by a cover pool of assets which typically consist of mortgage loans or public sector debt to which investors have a preferential claim in the event of default. Their secured nature and certain additional safety features, such as the requirement on the issuer to replace non-performing assets in the cover pool and maintain the cover pool at a value exceeding the par value of the bonds (‘asset coverage requirement’), have contributed to make covered bonds relatively low-risk, yield-bearing instruments with a key funding role in mortgage markets of most Member States. In certain Member States outstanding covered bond issuance exceeds the pool of outstanding government bonds. Certain covered bonds of credit quality step 1, in particular, exhibited an excellent liquidity performance during the period from 1 January 2008 to 30 June 2012 analysed by the EBA in its report. Nevertheless the EBA recommended treating thes

In [52]:
definitions_and_contexts[15]

[['1.   Level 2A assets shall only include assets falling under one or more of the following categories and meeting in each case the eligibility criteria laid down herein:',
  '(a)',
  'assets representing claims on or guaranteed by regional governments, local authorities or public sector entities in a Member State, where exposures to them are assigned a risk weight of 20 % in accordance with Article 115(1) and (5) and Article 116(1), (2) and (3) of Regulation (EU) No 575/2013, as applicable;',
  '(b)',
  'assets representing claims on or guaranteed by the central government or the central bank of a third country or by a regional government, local authority or public sector entity in a third country, provided that they are assigned a 20 % risk weight in accordance with Articles 114(2), 115 or 116 of Regulation (EU) No 575/2013, as applicable;',
  '(c)',
  'exposures in the form of high quality covered bonds, which shall comply with all of the following requirements:',
  '(i)',
  'they 

In [40]:
def_context[5]

'e'

In [36]:
print( articles[-2] )

ANNEX I
Formulae for the determination of the liquidity buffer composition
1.
Credit institution shall use the formulae laid down in this Annex to determine the composition of their liquidity buffer in accordance with Article 17.
2.
Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to:
(a)
the level 1 asset amount; plus
(b)
the level 2A asset amount; plus
(c)
the level 2B asset amount;
minus the lesser of:
(d)
the sum of (a), (b), and (c); or
(e)
the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex.
3.
‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein:
(a)
an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets, excluding level 1 covered bonds, that would be held by the credit institution upon the unwind of any secured funding transaction, secured lending transa

In [17]:
definition_articles[5]

[False, False, True, False, False, False, False]

In [19]:
for sent, defin in zip(sentences, definitions):
    if defin:
        print(sent)
        print("\n")
        
        

Covered bonds are debt instruments issued by credit institutions and secured by a cover pool of assets which typically consist of mortgage loans or public sector debt to which investors have a preferential claim in the event of default. Their secured nature and certain additional safety features, such as the requirement on the issuer to replace non-performing assets in the cover pool and maintain the cover pool at a value exceeding the par value of the bonds (‘asset coverage requirement’), have contributed to make covered bonds relatively low-risk, yield-bearing instruments with a key funding role in mortgage markets of most Member States. In certain Member States outstanding covered bond issuance exceeds the pool of outstanding government bonds. Certain covered bonds of credit quality step 1, in particular, exhibited an excellent liquidity performance during the period from 1 January 2008 to 30 June 2012 analysed by the EBA in its report. Nevertheless the EBA recommended treating thes

In [23]:
definition_articles[9]

[False, False, True, False, False, False, False, False, False]

In [31]:
print(articles[9] )

Article 6
Composition of the liquidity buffer
In order to be eligible to form part of a credit institution's liquidity buffer, the liquid assets shall comply with each of the following requirements:
(a)
the general requirements laid down in Article 7;
(b)
the operational requirements laid down in Article 8;
(c)
the respective eligibility criteria for their classification as a level 1 or level 2 asset in accordance with Chapter 2.


In [7]:
'' in ';dfdf'

True

In [3]:
definitions_and_contexts[-2]

[['Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to:',
  '(a)',
  'the level 1 asset amount; plus',
  '(b)',
  'the level 2A asset amount; plus',
  '(c)',
  'the level 2B asset amount;',
  'minus the lesser of:',
  '(d)',
  'the sum of (a), (b), and (c); or',
  '(e)'],
 ['the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex.'],
 ['‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein:',
  '(a)'],
 ['an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets, excluding level 1 covered bonds, that would be held by the credit institution upon the unwind of any secured funding transaction, secured lending transaction, asset exchange or collateralised derivatives transaction that matures within 30 calendar days from the calculation date and where the credit institution 

In [18]:
annotate_article( articles[-2].split( "\n" ) )

['root▢ANNEX I',
 'root▢Formulae for the determination of the liquidity buffer composition',
 'root■1.▢1.',
 'root■1.▢Credit institution shall use the formulae laid down in this Annex to determine the composition of their liquidity buffer in accordance with Article 17.',
 'root■2.▢2.',
 'root■2.▢Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to:',
 'root■2.■(a)▢(a)',
 'root■2.■(a)▢the level 1 asset amount; plus',
 'root■2.■(b)▢(b)',
 'root■2.■(b)▢the level 2A asset amount; plus',
 'root■2.■(c)▢(c)',
 'root■2.■(c)▢the level 2B asset amount;',
 'root■2.■(c)▢minus the lesser of:',
 'root■2.■(d)▢(d)',
 'root■2.■(d)▢the sum of (a), (b), and (c); or',
 'root■2.■(e)▢(e)',
 'root■2.■(e)▢the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex.',
 'root■3.▢3.',
 'root■3.▢‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein:',
 'root

In [3]:
i=4
annotated_article=annotate_article( articles[i].split("\n") )
definition_articles[i]

#sanity check:

assert( len( annotated_article  ) == len( definition_articles[i] )  )

extract_definitions_and_contexts(  annotated_article, definition_articles[i] )


[['‘level 1 assets’ means assets of extremely high liquidity and credit quality as referred to in the second subparagraph of Article 416(1) of Regulation (EU) No 575/2013;'],
 ['‘level 2 assets’ means assets of high liquidity and credit quality as referred to in the second subparagraph of Article 416(1) of Regulation (EU) No 575/2013. level 2 assets are further subdivided into level 2A and 2B assets in accordance with Chapter 2 of Title II of this Regulation;'],
 ['‘liquidity buffer’ means the amount of liquid assets that a credit institution holds in accordance with Title II of this Regulation;'],
 ['‘reporting currency’ means the currency in which the liquidity items referred to in Titles II and III of Part Six of Regulation (EU) No 575/2013 must be reported to the competent authority in accordance with Article 415(1) of that Regulation;'],
 ['‘asset coverage requirement’ means the ratio of assets to liabilities as determined for credit enhancement purposes in relation to covered bon

In [36]:
articles[0]

'test'

In [ ]:
assert( len(sentences) == len(definitions)  )

definitions_text=[]
for sentence, definition in zip(sentences, definitions):
    if definition:
        definitions_text.append( sentence )

for sentence in definitions_text:
    print(sentence)   

In [3]:


#send to annotation...

In [ ]:
for 

In [18]:
len( "".split("\n") )

1

In [33]:
articles=[ "test",  "test \n test"   , ""   ]
sentences="\n".join(articles)
sentences=sentences.split( "\n" )
sentences
definitions=[ True, True, False, False  ]
article_lengths=[len(article.split("\n") ) for article in articles]
index=0
definition_articles=[]
for length in article_lengths:
    definition_articles.append(definitions[index: index+length ])
    index=index+length



In [34]:
definition_articles

[[True], [True, False], [False]]

In [16]:
print(articles[-1])

ANNEX II
Formula for the calculation of the net liquidity outflow
NLO
=
Net liquidity outflow
TO
=
Total outflows
TI
=
Total inflows
FEI
=
Fully exempted inflows
IHC
=
Inflows subject to higher cap of 90 % outflows
IC
=
Inflows subject to cap of 75 % of outflows
Net liquidity outflows equals total outflows less the reduction for fully exempt inflows less the reduction for inflows subject to the 90 % cap less the reduction for inflows subject to the 75 % cap
NLO = TO – MIN(FEI, TO) – MIN(IHC, 0,9*MAX(TO – FEI, 0)) – MIN(IC, 0,75*MAX(TO – FEI – IHC/0,9, 0))


In [15]:
definition_articles[-1]

[False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True]

In [8]:
for definition, sentence in zip( definition_articles[4], articles[4].split( "\n" )  ):
    if definition:
        print(sentence)

‘level 1 assets’ means assets of extremely high liquidity and credit quality as referred to in the second subparagraph of Article 416(1) of Regulation (EU) No 575/2013;
‘level 2 assets’ means assets of high liquidity and credit quality as referred to in the second subparagraph of Article 416(1) of Regulation (EU) No 575/2013. level 2 assets are further subdivided into level 2A and 2B assets in accordance with Chapter 2 of Title II of this Regulation;
‘liquidity buffer’ means the amount of liquid assets that a credit institution holds in accordance with Title II of this Regulation;
‘reporting currency’ means the currency in which the liquidity items referred to in Titles II and III of Part Six of Regulation (EU) No 575/2013 must be reported to the competent authority in accordance with Article 415(1) of that Regulation;
‘asset coverage requirement’ means the ratio of assets to liabilities as determined for credit enhancement purposes in relation to covered bonds by the national law of a

In [4]:
print(articles[4])

Article 3
Definitions
For the purposes of this Regulation, the following definitions shall apply:
(1)
‘level 1 assets’ means assets of extremely high liquidity and credit quality as referred to in the second subparagraph of Article 416(1) of Regulation (EU) No 575/2013;
(2)
‘level 2 assets’ means assets of high liquidity and credit quality as referred to in the second subparagraph of Article 416(1) of Regulation (EU) No 575/2013. level 2 assets are further subdivided into level 2A and 2B assets in accordance with Chapter 2 of Title II of this Regulation;
(3)
‘liquidity buffer’ means the amount of liquid assets that a credit institution holds in accordance with Title II of this Regulation;
(4)
‘reporting currency’ means the currency in which the liquidity items referred to in Titles II and III of Part Six of Regulation (EU) No 575/2013 must be reported to the competent authority in accordance with Article 415(1) of that Regulation;
(5)
‘asset coverage requirement’ means the ratio of ass

In [39]:
len( articles[-1].split("\n") )

22

In [4]:
sum(definitions)

12

In [ ]:
definition_finder=DefinitionFinder( [ '‘this’ means that' , 'are you ok'  ] )
definition_finder.get_definition_regex()

[True, False]